In [110]:
import json
import urllib
from urllib.request import Request, urlopen
import pandas as pd
import numpy as np
from tqdm import notebook
import plotly.graph_objects as go
import plotly.express as px
import ast 
import warnings
warnings.filterwarnings(action='ignore')

In [111]:
station = pd.read_csv('data/seoul_bike_station_01_12.csv',encoding="CP949",index_col=0)
near_bus = pd.read_csv('data/near_bus_200m.csv',encoding="CP949",index_col=0)
seoul_bike = pd.read_parquet('D:/git_local_repository/data_for_project/seoul_bike/seoul_bike_2021.parquet.gzip',columns=[ 'st_id1', 'st_id2', 'riding_time',
       'dist', 'm_pm'])

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2

    c = 2 * np.arcsin(np.sqrt(a))
    m = 6367 * c * 1000
    return m

> ### 이름으로 버스 정류장과 대여소 선택

In [168]:
# 출발, 도착
starting_point = near_bus[near_bus['name'].str.contains('염창역')] # 버스, 대여소, 역 모두 넣어놓는 리스트로 만들기
ending_bike = 754
ending_point = station.query("id==@ending_bike")

def finding_start_end(starting_point,ending_point) :
    # 목적지 대여소와 가까운 버스 정류장 검색 -> bus_route_near_bike
    result_km = haversine_np(
        ending_point["경도"].values,
        ending_point["위도"].values,
        near_bus["경도"].values,
        near_bus["위도"].values,
    )
    near_bus_for_dist = near_bus.copy()
    near_bus_for_dist["dist"] = result_km
    bus_route_near_bike = (
        near_bus_for_dist[near_bus_for_dist["dist"] < 1500]
        .dropna()
        # .drop_duplicates(subset=["id"])
        .sort_values(by="id")
    )
    # 겹차는 노선 찾기
    bus_route_inter = np.intersect1d(
        bus_route_near_bike["bus"].unique(), starting_point["bus"].unique()
    ).tolist()

    # 검색 결과 없을때 검색 종료
    if bus_route_inter == [] : 
        print('검색결과 없음')
        # 정류장,지하철역, 대여소 등 관련 내용이 없으면 station => station 검색 함수로 옮김
        return print('station_to_station')

    # 겹치는 노선만 들어있는 버스정류장
    starting_bus = starting_point.query("bus == @bus_route_inter")
    ending_bus = bus_route_near_bike.query("bus == @bus_route_inter")
    

    # 정류장 시작,종료 지점 모으기.. 머리 간질간질해 죽는줄..
    bus_df = pd.DataFrame(columns=ending_bus.columns)
    for num in bus_route_inter:
        starting_order = starting_bus.query("bus==@num")
        ending_order = ending_bus.sort_values(by=["dist",'order']).query("bus==@num")[:2]

        # 하행인 경우 ending.max - starting.max > 0 이며 상행인 경우 그 반대임.

        #하행노선 찾기
        if ending_order['order'].max() - starting_order['order'].max() >= 0 :
            df = starting_order.sort_values(by='order',ascending=False)[:1]
            bus_df = bus_df.append(df)
            df = ending_order.sort_values(by='order',ascending=False)[:1]
            bus_df = bus_df.append(df)
            print('후아')

        #상행노선 찾기
        elif ending_order['order'].max() - starting_order['order'].max() < 0 :
            df = starting_order.sort_values(by='order')[:1]
            bus_df = bus_df.append(df)
            df = ending_order.sort_values(by='order')[:1]
            bus_df = bus_df.append(df)
            print('머지')

        
    # 시작 위치가 1.5km이내에 있는 경우 버스 경로검색 안하도록 조치
    if starting_order['id'].iloc[0] in bus_route_near_bike['id'].tolist() :
        bus_df = starting_order.sort_values(by='order',ascending=False)[:1]
        print('졸리')
        # 아래 함수랑 연동시켜보자
        return bus_df        

    # 아래 함수랑 연동시켜보자
    return bus_df


pick_destination=finding_start_end(starting_point, ending_point)

# 추천 dist 가까운 순으로 하기 
# 만약 112클릭하면 111,112,113 버스정류장과 근처 따릉이 지도에 표시하기 
# 112와 그 주변 따릉이 색은 다른곳과 다르게 찐하게 표시하기
pick_destination

후아
후아
졸리


,bike_id,bus,order,id,name,경도,위도
1391,"[773,1169]",N62,138,16001,염창역.서울도시가스,126.875881,37.547097


In [151]:
pick = pick_destination[:2]

pick

,bike_id,bus,order,id,name,경도,위도,dist
523,"[104,105,106]",N26,108,14012,합정역,126.913997,37.549682,NaN
527,"[773,1169]",N26,112,16001,염창역.서울도시가스,126.875881,37.547097,665.465568


In [152]:
def bus_route_info(pick): 
    if pick.iloc[-1]["id"] == pick.iloc[0]["id"]:
        print("버스를 타지 않았네")
        return [],[],[],[]
    else:
        # 범위선택
        bus_order_start = pick["order"].iloc[0]
        bus_order_end = pick["order"].iloc[-1]
        bus_name = pick["bus"].iloc[0]

        # 자료 sorting
        route_whole = near_bus.query(
            "@bus_order_start <= order <= @bus_order_end & bus == @bus_name"
        )[["name", "경도", "위도",'order']].sort_values(by='order')
        route_start = route_whole.iloc[0]
        route_end = route_whole.iloc[-1]
        route_start_and_end = pd.concat([route_start,route_end],axis=1).T
        route_middle = route_whole.iloc[int(len(route_whole) / 2)]

        #return
        bus_route = [route_start_and_end, route_middle, route_whole]

    return bus_route

route_start_and_end, route_middle, route_whole = bus_route_info(pick)
# return으로 bus route를 받으면 될듯
# route end name은 출발, 끝 지점 텍스트 표시할때 사용하면 될듯

In [156]:
## 외부 함수
# 위에있는 데이터프레임이 들어가야함

def bike_to_bike(bike_id: list[int] , ending_bike_id:int) :

    # 추천_대여소_선정 : ast : '[123,]' => [123]으로 만들어줌 --> list
    if type(bike_id) == str :
        bike_id = ast.literal_eval(bike_id)
    

    # 추천 대여소 리스트 만들기 
    BM_1 = (seoul_bike['st_id1'] == ending_point['id'].iloc[0]) & (seoul_bike['st_id2'].isin(bike_id))
    BM_2 = (seoul_bike['st_id2'] == ending_point['id'].iloc[0]) & (seoul_bike['st_id1'].isin(bike_id))
    seoul_bike_sorting = seoul_bike.loc[BM_1 | BM_2] # 거리 및 시간 계산을 위해 따로 추출

    # 버스정류장 근처 대여소와 집 근처 대여소 거리 비교
    ending_station = ending_point['id'].iloc[0]

    bike_recommend =[]
    for starting_station in bike_id:
        BM_1 = (seoul_bike_sorting["st_id1"] == starting_station) & (seoul_bike_sorting["st_id2"] == ending_station)
        BM_2 = (seoul_bike_sorting["st_id2"] == starting_station) & (seoul_bike_sorting["st_id1"] == ending_station)

        # 평균거리 계산
        mean_station = (
            seoul_bike_sorting[BM_1 | BM_2]["riding_time"]
            .value_counts()
            .sort_values(ascending=False)[:5]
            .index.values.mean()
        )

        # 이용기록 계산
        num_station = len(seoul_bike_sorting[BM_1 | BM_2])

        # 직선거리 계산
        lat_1 = station.query("id==@starting_station")["위도"].values
        lon_1 = station.query("id==@starting_station")["경도"].values
        lat_2 = station.query("id==@ending_station")["위도"].values
        lon_2 = station.query("id==@ending_station")["경도"].values
        dist_station = haversine_np(lat_1, lon_1, lat_2, lon_2)
        
        #자료 저장
        station_info = station.query('id==@starting_station')
        bike_recommend.append([station_info['주소2'].iloc[0],station_info['위도'].iloc[0],station_info['경도'].iloc[0],starting_station, mean_station,num_station,f"{dist_station[0]/1000 :.2f}km"])

    #return 값    
    bike_recommend = pd.DataFrame(bike_recommend,columns=['주소2','위도','경도','id','time','num','dist'])

    return bike_recommend



# 16001 염창
# 13045 충정로
# starting_bus_id = 16002
ending_bike_id = 754
bike_id = pick.iloc[-1]['bike_id'] # bike_id는 무조건 리스트 형태로
bike_recommend = bike_to_bike(bike_id, ending_bike_id)

bike_recommend



,주소2,위도,경도,id,time,num,dist
0,"염창역 3, 4번 출구 사이",37.546707,126.874573,773,8.0,723,0.44km
1,염창역 1번 출구,37.547199,126.874489,1169,9.0,231,0.47km


In [172]:
# trick figure
fig = px.scatter_mapbox(station.query("id == 207"), lat="위도", lon="경도",zoom=12)
fig.update_traces(
    marker=go.scattermapbox.Marker(symbol="circle", size=10, opacity=0.001, color="white")
)

fig.update_layout(
    mapbox=dict(
        accesstoken="pk.eyJ1IjoieWFuZ29vcyIsImEiOiJjbDNqd2tkN2IwbGdmM2pvNzF0c2M4NnZkIn0.J3IjPYg3w28cGiWkUD7bnA",
        # style='mapbox://styles/yangoos/cl3jubvl7000c14llgtoev0nm'
    ),
    margin=dict(l=20, r=20, t=20, b=20)
)
fig.show()

In [169]:
# trick figure
fig = px.scatter_mapbox(station.query("id == 207"), lat="위도", lon="경도")
fig.update_traces(
    marker=go.scattermapbox.Marker(symbol="circle", size=10, opacity=0.001, color="white")
)


# -- 추천 대여소 그리기 -- #
recomend_id = result_recommend['id'].tolist()
fig = px.scatter_mapbox(station.query("id == @recomend_id"), lat="위도", lon="경도")
fig.update_traces(
    marker=go.scattermapbox.Marker(symbol="circle", size=10, opacity=0.7, color="blue")
)


# -- 추천 경로 그리기 -- # 
def get_optimal_route(start, goal, waypoints=["", ""], option="traoptimal"):
    client_id = '44vlsxxinm'
    client_secret = 'BOHC6psFnLkjdRrlkLn24S1apLUTsuzrCnMuCi4A'
    # start=/goal=/(waypoint=)/(option=) 순으로 request parameter 지정
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction-15/v1/driving?start={start[0]},{start[1]}&waypoint={waypoints[0]},{waypoints[1]}&goal={goal[0]},{goal[1]}&option={option}"
    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID", client_id)
    request.add_header("X-NCP-APIGW-API-KEY", client_secret)

    response = urllib.request.urlopen(request)
    res = response.getcode()

    if res == 200:
        response_body = response.read().decode("utf-8")
        return json.loads(response_body)

    else:
        print("ERROR")


#변수 
# option : 탐색옵션 [최대 3개, traoptimal(기본 옵션)
# / trafast, tracomfort, traavoidtoll, traavoidcaronly]
option = "traoptimal"
starting_bus = route_start_and_end[['경도','위도']].values[0]
ending_bus = route_start_and_end[['경도','위도']].values[1]
way_bus  = route_middle[['경도','위도']].values

result = get_optimal_route(starting_bus, ending_bus,way_bus)


a = result.get('route').get(option)[0].get('path')
b = pd.DataFrame(a)

fig_1 = go.Figure(
    go.Scattermapbox(
        lat=b[1],
        lon=b[0],
        mode="lines",
        marker=dict(symbol="circle", size=15, color='#AD00FF' , opacity=0.5),
    )
)
fig.add_trace(fig_1.data[0])


# -- bus 정류장 그리기 --#
fig_2 = px.scatter_mapbox(route_start_and_end, lat="위도", lon="경도", hover_name="name",zoom=11,height=500
)
fig_2.update_traces(
    marker=go.scattermapbox.Marker(symbol="circle", size=10, opacity=0.7, color="#AD00FF")
)
fig.add_trace(fig_2.data[0])


# --반경 내 따릉이 대여소 그리기--#

# 거리 계산
station_haversine = haversine_np(
    pick.iloc[-1]["경도"],
    pick.iloc[-1]["위도"],
    station["경도"].values,
    station["위도"].values,
)

# 데이터 추가
around_station = station.copy()
around_station['dist'] = station_haversine
around_station = around_station[around_station['dist'] < 500] 

# 시각화
fig_3 = go.Figure(
    go.Scattermapbox(
        name=f"station with in {round(500/1000,2)}km",
        lat=around_station["위도"],
        lon=around_station["경도"],
        mode="markers",
        # marker = {'size': 20, 'symbol': "bus"},
        marker=go.scattermapbox.Marker(size=8, color="black", opacity=0.2),
        text=around_station["주소2"],
    )
)
fig.add_trace(fig_3.data[0])

# --목적지 대여소 그리기 -- #
fig_4 = go.Figure(
    go.Scattermapbox(
        name="location",
        lat=ending_point["위도"],
        lon=ending_point["경도"],
        mode="markers",
        # marker = {'size': 20, 'symbol': "bus"},
        marker=go.scattermapbox.Marker(size=10, color="fuchsia", opacity=0.8),
        text=ending_point["주소2"].values,
    )
)
fig.add_trace(fig_4.data[0])

fig.update_layout(
    mapbox=dict(
        accesstoken="pk.eyJ1IjoieWFuZ29vcyIsImEiOiJjbDNqd2tkN2IwbGdmM2pvNzF0c2M4NnZkIn0.J3IjPYg3w28cGiWkUD7bnA",
        # style='mapbox://styles/yangoos/cl3jubvl7000c14llgtoev0nm'
    ),
    margin=dict(l=20, r=20, t=20, b=20)
)
fig.show()


In [ ]:
ending_point  = station.query("id == @ending_bike_id")[['경도','위도']].values[0]

starting_point = result_recommend[['경도','위도']].values[0]


126.874489

> ### 길찾기 기능(비추..)

In [ ]:
# *-- Directions 5 활용 코드 --*





fig.update_layout(
    mapbox=dict(
        accesstoken="pk.eyJ1IjoieWFuZ29vcyIsImEiOiJjbDNqd2tkN2IwbGdmM2pvNzF0c2M4NnZkIn0.J3IjPYg3w28cGiWkUD7bnA",
        center=go.layout.mapbox.Center(
            lat=37.58,
            lon=126.87
        ),
        # pitch=0,
        zoom=10
        # style='mapbox://styles/yangoos/cl3jubvl7000c14llgtoev0nm'
    ),
    margin=dict(l=20, r=20, t=20, b=20)
)
fig.show()




> ### 자전거 대여소 간 계산
* 해당 경우 검색 시 
* 지하철 -> 대여소 -> 대여소
* 대여소 -> 대여소 

In [ ]:


# 345ms
def st_to_st_new (starting_station, ending_station):

    lat_1=station.query('id==@starting_station')['위도'].values
    lon_1=station.query('id==@starting_station')['경도'].values
    lat_2=station.query('id==@ending_station')['위도'].values
    lon_2=station.query('id==@ending_station')['경도'].values
    dist_station = haversine_np(lat_1,lon_1,lat_2,lon_2)

    BM_1 = (seoul_bike['st_id1'] == starting_station) & (seoul_bike['st_id2'] == ending_station)
    BM_2 = (seoul_bike['st_id2'] == starting_station) & (seoul_bike['st_id1'] == ending_station)

    # 평균거리 계산
    sort_station = seoul_bike[BM_1 | BM_2]
    mean_station = sort_station[sort_station['dist'] < dist_station[0]*2]['riding_time'].value_counts()[:5].index.values.mean()
    num_station = len(sort_station)

    print(mean_station,num_station,f'{dist_station[0]/1000 :.2f}km' )

st_to_st_new(207, 754)


In [ ]:
## 옛날자료

def st_to_st(starting_station, ending_station):

    """
    starting_station = list or val이고 bike_station id가 들어감
    ending_station = list or val이고 bike_station id가 들어감
    """

    # int 또는 list로 starting station 받기
    if type(starting_station) == int:
        starting_station_list = list()
        starting_station_list.append(starting_station)
    else:
        starting_station_list = starting_station

    for starting_station in starting_station_list:
        BM_1 = (seoul_bike_sorting["st_id1"] == starting_station) & (
            seoul_bike_sorting["st_id2"] == ending_station
        )
        BM_2 = (seoul_bike_sorting["st_id2"] == starting_station) & (
            seoul_bike_sorting["st_id1"] == ending_station
        )

        # 평균거리 계산
        mean_station = (
            seoul_bike_sorting[BM_1 | BM_2]["riding_time"]
            .value_counts()
            .sort_values(ascending=False)[:5]
            .index.values.mean()
        )

        # 이용기록 계산
        num_station = len(seoul_bike_sorting[BM_1 | BM_2])

        # 직선거리 계산
        lat_1 = station.query("id==@starting_station")["위도"].values
        lon_1 = station.query("id==@starting_station")["경도"].values
        lat_2 = station.query("id==@ending_station")["위도"].values
        lon_2 = station.query("id==@ending_station")["경도"].values
        dist_station = haversine_np(lat_1, lon_1, lat_2, lon_2)
        print(mean_station, num_station, f"{dist_station[0]/1000 :.2f}km")
    # return


st_to_st(counts_station, 754)
